In [1]:
from ase.calculators.vasp import Vasp2
from ase.io import read
import numpy as np
import logging
import os

!rm -r bench_Si_autotune_ENCUT


# Load Structure and Set Output Directory

In [ ]:
out_dir = 'bench_Si_autotune_ENCUT'
sys = read("POSCAR.mp-27_Si")

# Start Log and Define Calc

In [10]:
if not os.path.isdir(out_dir+"/log"):
    os.system("mkdir -p {}/log".format(out_dir))
logging.basicConfig(format='%(asctime)s : %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p',
                   filename=out_dir+"/log/valet.log", filemode="w",level=logging.INFO)
calc = Vasp2(xc='PBE',kpts=(4,4,4),directory=out_dir,atoms=sys)

# Methods

In [13]:
def check_stop(x,tol):
    perc_diff = lambda x,y: np.abs(np.abs(x-y)/((x+y)/2))*100
    if len(x) < 2:
        return False
    elif perc_diff(x[-1],x[-2]) < tol:
        return True
    else: 
        return False

def set_encut_autotune_params(tol=1E-4,max_steps=15,start_encut=200,encut_step=50):
    stop = False
    ens = []
    cnt = 0
    return tol,stop,cnt,max_steps,start_encut,encut_step,ens


def do_encut_autotune(tol=1E-4,max_steps=15,start_encut=200,encut_step=50):
    tol,stop,cnt,max_steps,ENCUT,ENCUT_step,ens = set_encut_autotune_params(tol=tol,max_steps=max_steps,
                                                                                 start_encut=start_encut,
                                                                                 encut_step=encut_step)
    logging.info("Commencing with ENCUT autotune.")
    print("Autotuning ENCUT...")
    while not stop:
        if 'error' in locals():
            del error
        if cnt < max_steps:
            logging.debug("Count {}".format(cnt))
            calc.set(encut=ENCUT)
            ens.append(sys.get_potential_energy())
            #print(ENCUT)
            stop = check_stop(ens,tol)
            ENCUT += ENCUT_step
            cnt += 1
        else:
            logging.debug("Maximum autotune steps reached.")
            error = "Maximum autotune steps reached."
            tip = "Tip: Increase either start_encut, encut_step, or max_steps. Decrease tol."
            stop = True

    os.system("rm {}/*".format(out_dir))
    logging.info("Completed ENCUT autotune.")
    try:
        print(error)
        print(tip)
        return None
    except NameError:
        print("\tENCUT auto-tuned to: {}".format(ENCUT))
        return ENCUT
    

In [14]:
ENCUT = do_encut_autotune(tol=1E-4,start_encut=800)
print(ENCUT)

Autotuning ENCUT...
	ENCUT auto-tuned to: 900
900
